In [2]:
import numpy as np
import cv2
import scipy as sc
# import matplotlib.pyplot as plt
from glob import glob 
import re
import os
import sys

# adds the lower lying directory to the import path to import the other modules
Lpath = os.path.abspath('..')
print("found path with os.path.abspath('..'): ", Lpath)
sys.path.insert(0, Lpath)

import tflearn
from face_detect import cutFace
from showNumpyInfo import showInfo

import logging
from memory_profiler import memory_usage



# if __name__ == "__main__":
logging.basicConfig(level=logging.DEBUG, filename="logfile", filemode="a+",
                    format="%(asctime)-15s %(levelname)-8s %(message)s")
logging.info("hello")

def logprint(stre):
	print(stre)
	logging.info(stre)
    
    

found path with os.path.abspath('..'):  G:\Documenten\personal\school\MaNaMA_AI\thesis\implementation\dexpression\github_1\github\Thesis_Christiaan_Jean\Custom_Dexpression


In [9]:

dataPath = 'G:/Documenten/personal/school/MaNaMA_AI/thesis/databases/wikipedia_list/cohn-Kanade/CK+'
datasetPath = dataPath
printData = 1
cascPath = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/DeXpression-master_chris/haarcascade.xml"
allData = False,
neutral = True

In [10]:
#stores the current working directory path to restore at the end and goes to the given database path
curr_path = os.getcwd()
os.chdir(datasetPath)
#get a list of URLs to all the emotion file that can be found
label_list = glob('./Emotion/*/*/*.txt')

X_data = [] # list of input data = climax images of emotions
Y_data = [] # list of output data = emotion expressed in image
X_subID = [] # list with subject Id of each X_data element 

# each emotion clip has only a few relevant images that represent an emotion 
relevant_part = 0.33 # the "precentage" of the end of the clip that has relevant images for that emotion


i = 0 #count of total amount of instance
tot_img_count = 0 # Count of how many images there are loaded

#count per emotion
N = 0 # Neutral
A = 0 # Anger
C = 0 # Contempt
D = 0 # Disgust
F = 0 # Fear
H = 0 # Happy
Sa = 0# Saddness
Su = 0# Surprise

#subject counter
sub = 0
lastSub = 0
subjectID = 0

#count of dimensions
dimMap = {}


In [11]:


# # image
# img = cv2.imread(str)

print("begin loading data")
#iterate through the list of label files, open corresponding images
for fn in label_list:

    str = './cohn-kanade-images'+ fn[9:-29] + '*.png'

    # print(fn[9:-29])
    # print(str)

    img_list = glob(str)


    img = []
    img_neutral =[]

    relevant_amount = int(round(len(img_list)*relevant_part))

    # stre = "from " + repr( len(img_list)) +  "amount of images only " + repr(relevant_amount) + " is relevant"
    # logging.info(stre)
    tot_img_count = tot_img_count + relevant_amount 
    # print("tot_img_count ", tot_img_count)


    #the first 3 images are taken as being neutral
    count = 0
    for url in img_list[0:3]:
        # print(url)
        image = cv2.imread(url)
        img_neutral.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        N = N + 1
        # stre = "neutral img = " + repr(count)
        # cv2.imshow(stre, image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        count = count +1

    # relevant amount of  the last frames are taken as belonging to that emotion
    count = 0
    for url in img_list[(len(img_list)-relevant_amount) :len(img_list)]:
        # print(url)
        image = cv2.imread(url)
        img.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

        # stre = "emotion img = " + repr(count)
        # cv2.imshow(stre, image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        count = count +1


    #sizeStr = repr(img.shape[0]) + "x" + repr(img.shape[1]) #+ "x" + repr(img.shape[2])

    # test accessing single elements
    # if sizeStr in dimMap:
    # 	dimMap[sizeStr] = dimMap[sizeStr] + 1 
    # else:
    # 	dimMap[sizeStr] = 1

    # read emotion labels from the file
    file = open(fn,'r')
    fileTxt = file.read()
    emotionNr = int(fileTxt[3:4])
    file.close()

    #determine the subject (number is within URL)
    subjectID = fn[11:14]
    if subjectID != lastSub :
        sub = sub + 1
    lastSub = subjectID 

    #determine the emotion of the image
    if(emotionNr == 0):
        emotion = 'Neutral'
        N = N + relevant_amount
    elif(emotionNr == 1):
        emotion = 'Anger'
        A = A + relevant_amount 
    # elif(emotionNr == 2):
    # 	emotion = 'Contempt'
    # 	C = C + relevant_amount 
    elif(emotionNr == 3):
        emotion = 'Disgust'
        D = D + relevant_amount 
    elif(emotionNr == 4):
        emotion = 'Fear '
        F = F + relevant_amount 
    elif(emotionNr == 5):
        emotion = 'Happy'
        H = H + relevant_amount 
    elif(emotionNr == 6):
        emotion = 'Saddness'
        Sa = Sa + relevant_amount 
    else:
        emotion = 'Surprise'
        Su = Su + relevant_amount 



    if(emotionNr != 2):
        # store images of the neutral expression and emotion label in the X_data and Y_data lists
        for j in img_neutral:
            X_data.append(j)
            Y_data.append(2)
            X_subID.append(subjectID)	

        # store image and emotion label in the X_data and Y_data lists
        for j in img:
            X_data.append(j)
            Y_data.append(emotionNr)
            X_subID.append(subjectID)

        # print("X_data length ",len(X_data))

        i = i+len(img) + len(img_neutral)

    if (sub%20 ==0):
        logprint(repr(sub)+"subjects loaded")
        mem_usage = memory_usage(-1, interval=.2, timeout=1)
        str = 'Maximum memory usage: ' + repr( mem_usage)
        logprint(str)

    if printData :
        logging.info("-------------------------------")

        logging.info(str) #./cohn-kanade-images/S506/002/S506_004_00000038.png
        logging.info("amount of image added " + repr( len(img)))
        logging.info("image type:   " + repr(type(img)))

       


        logging.info('image ' + repr(i) + " " + emotion)

#show stattistics
logging.info("--------- Overal stattistics ---------  ")
logging.info("amount of Subjects  : " + repr(sub))
logging.info("amount of Instances : " + repr(i))
logging.info("code = 1 = Anger      " + repr(A) + " instances: " + repr(np.round((float(A)/i)*100),decimals=2))
logging.info("code = 2 = Neutral    " + repr(N) + " instances: " + repr(np.round((float(N)/i)*100),decimals=2))
logging.info("code = 3 = Disgust    " + repr(D) + " instances: "+ repr(np.round((float(D)/i)*100),decimals=2))
logging.info("code = 4 = Fear       " + repr(F) + " instances: "+ repr(np.round((float(F)/i)*100),decimals=2))
logging.info("code = 5 = Happy      " + repr(H) + " instances: "+ repr(np.round((float(H)/i)*100),decimals=2))
logging.info("code = 6 = Saddness   " + repr(Sa) +" instances: "+ repr(np.round((float(Sa)/i)*100),decimals=2))
logging.info("code = 7 = Surprise   " + repr(Su) +" instances: "+ repr(np.round((float(Su)/i)*100),decimals=2))
logging.info("--------- dimensions ---------  ")
# print(repr(dimMap))

logging.info("--------- last elements in lists ---------  ")
logging.info("length X_data " + repr(len(X_data))) 
logging.info("length Y_data " + repr(len(Y_data)))
logging.info("length X_subID " + repr(len(X_subID))) 


# # cv2.imshow('image',X_data[len(X_data)-1])
# plt.figure('last image')
# plt.imshow(X_data[len(X_data)-1], cmap='gray')#, interpolation='nearest');
# plt.show()

print("length Y_data " + repr(Y_data[len(Y_data)-1])) 
print("image type " + repr(type(Y_data[len(Y_data)-1]))) 

mem_usage = memory_usage(-1, interval=.2, timeout=1)
str = 'Maximum memory usage: ' + repr( mem_usage)
logprint(str)




os.chdir(curr_path)

begin loading data
20subjects loaded
Maximum memory usage: [277.5, 277.53515625, 277.53515625, 277.53515625, 277.53515625]
20subjects loaded
Maximum memory usage: [281.44921875, 281.44921875, 281.44921875, 281.44921875, 281.44921875]
40subjects loaded
Maximum memory usage: [462.27734375, 462.28125, 462.28125, 462.28125, 462.28125]
40subjects loaded
Maximum memory usage: [466.5859375, 466.58984375, 466.58984375, 466.58984375, 466.58984375]
60subjects loaded
Maximum memory usage: [618.6015625, 618.6015625, 618.60546875, 618.60546875, 618.60546875]
60subjects loaded
Maximum memory usage: [621.20703125, 621.20703125, 621.2109375, 621.2109375, 621.2109375]
80subjects loaded
Maximum memory usage: [740.5625, 740.5625, 740.5625, 740.5625, 740.5625]
80subjects loaded
Maximum memory usage: [744.95703125, 744.95703125, 744.95703125, 744.95703125, 744.95703125]
80subjects loaded
Maximum memory usage: [747.7578125, 747.76171875, 747.76171875, 747.76171875, 747.76171875]
100subjects loaded
Maximum m

In [14]:
#bincount 
np.bincount(np.asarray(Y_data))

array([  0, 340, 927, 291, 183, 444, 182, 441], dtype=int64)

In [ ]:
# if(allData==True):
#     data = load_all_annotated_CKP_data(datasetPath,printData)
# elif(neutral == True):
#     data = load_all_annotated_CKP_data_neutral(datasetPath,printData)
# else:		
#     data = load_CKP_data(datasetPath,printData)

# save part of the data in temp.npy files
amount_sets = len(data)//50

if not os.path.exists('./temp'):
    os.makedirs('./temp')
    print("made new directory: " + './temp')

start = 0    
for i in range(0,amount_sets-1):
    stre = "./temp/temp_" + repr(i) + "_" + repr(start) + "-" + repr(start+49) + ".npy"
    npy.save(stre,data[start:start+50])
    start = start +50

stre = "temp_" + repr(i) + "_" + repr(start) + "-" + repr(len(data)) + ".npy"
npy.save(stre,data[start:])

mem_usage = memory_usage(-1, interval=.2, timeout=1)
str = 'Maximum memory usage before data = null: ' + repr( mem_usage)
logprint(str)

data = None;

mem_usage = memory_usage(-1, interval=.2, timeout=1)
str = 'Maximum memory usage after data = null: ' + repr( mem_usage)
logprint(str)



images = data[0]

# cut faces from each images and resize (downsample) to input dimension
X_gray = []
X_test = []

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)
stre ="total amount of images is "+ repr( len(images) )
logprint(stre)
logprint("start cutting the faces")

list_files = glob("./temp/*")

for file in list_files:
    logprint("load file: " + file)
    images = np.load(file)

    for i in range(0,len(images)):	   
        cut_img = cutFace(images[i],224,224,faceCascade)
        X_gray = np.append(X_gray,cut_img)
        if(i%50 == 0):
            stre =" "+ repr(i) + " images have been processed"
            print(stre)
            logging.info(stre)
            #monitor the memory usage of the current proces for 1 second, sample each 0.2 s return list of MB values
            mem_usage = memory_usage(-1, interval=.2, timeout=1)
            str = 'Maximum memory usage: ' + repr( mem_usage)
            logprint(str)

        if printData==1 :
#             cv2.imshow("example", cut_img.reshape((224,224)))
#             cv2.waitKey(0)

X_gray = X_gray.reshape([-1,224,224,1])
print(X_gray.shape)

# reformat the emotionlabels to create a vector of 7 zeros only one of these elements is set to 1 indicating the emotion label
labels = []

logprint("start coverting Y_class_data to output data")
for lab in data[1]:
    inst = np.zeros(7)
    inst[lab-1] = 1
    labels = np.append(labels,inst)

print("size Y 2: " + repr(labels.shape))

labels = labels.reshape([-1,7]) 

print("size labels flatten: " + repr(len(labels)))
print("size labels shape: " + repr(labels.shape))
print("type labels: " + repr(type(labels)))

outputData = [X_gray,labels,data[2]]

logprint("the end")


In [ ]:

if not os.path.exists('../data/CKP_all_neutral'):
    os.makedirs('../data/CKP_all_neutral')
    print("made new directory: " + ' ../data/CKP_all_neutral')

np.save('../data/CKP_all_neutral/X.npy',data[0])
np.save('../data/CKP_all_neutral/Y.npy',data[1])
np.save('../data/CKP_all_neutral/subjectIDs.npy',data[2])



In [ ]:
create_all_CKP_formated_data_neutral(dataPath,1)